In [ ]:
from jenga.corruptions import DuckDBCorruptionBrokenCharacters, BrokenCharacters
import pandas as pd
import duckdb
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_json("./data/Grocery_and_Gourmet_Food_5.json", lines=True)
df

In [ ]:
random.seed(4521)
seeds = [random.random() for _ in range(10)]
seeds

In [ ]:
duck_computing_time = []
duckdb.sql("SET threads = 1")
for seed in seeds:
    duckdb.sql(f"SELECT setseed({seed})")
    duckdb_corruption = DuckDBCorruptionBrokenCharacters("reviewText", 0.2)
    start = time.time()
    duckdb_corruption.transform(df)
    end = time.time()
    duck_computing_time.append(end - start)
duck_computing_time

In [ ]:
# This were the results of the experiment
duck_computing_time = [
    56.379162311553955,
    45.79448199272156,
    53.21978259086609,
    49.483625173568726,
    48.91889500617981,
    52.05965709686279,
    47.06034827232361,
    45.97737526893616,
    49.353299617767334,
    49.73344278335571
]

In [ ]:
duckdb_multiple_threads_computing_time = []
duckdb.sql("SET threads = 4")
for seed in seeds:
    duckdb.sql(f"SELECT setseed({seed})")
    duckdb_corruption = DuckDBCorruptionBrokenCharacters("reviewText", 0.2)
    start = time.time()
    duckdb_corruption.transform(df)
    end = time.time()
    duckdb_multiple_threads_computing_time.append(end - start)

duckdb_multiple_threads_computing_time

In [ ]:
duckdb_multiple_threads_computing_time = [
    58.86254072189331,
    47.50891900062561,
    47.77925133705139,
    49.86981225013733,
    45.86521124839783,
    50.89463400840759,
    47.12282633781433,
    50.97922658920288,
    51.78722262382507,
    53.28171372413635
]

In [ ]:
computing_time_jenga = []
for seed in seeds:
    random.seed(seed)
    duckdb_corruption = BrokenCharacters("reviewText", 0.2)
    start = time.time()
    duckdb_corruption.transform(df)
    end = time.time()
    computing_time_jenga.append(end - start)
    print(end - start)
# Here the result of experiments done on the machine 
computing_time_jenga

In [ ]:
computing_time_jenga = [
    105.71946358680725,
    112.38954043388367,
    114.71290516853333,
    103.92811894416809,
    113.79899501800537,
    109.98125863075256,
    116.37976813316345,
    102.42627382278442,
    108.20121622085571,
    104.77867007255554
]

In [ ]:

df_results = pd.DataFrame(columns=["type", "timespan"])
df_results = pd.concat([
    df_results, 
    pd.DataFrame([ { "type": "Jenga", "timespan": t} for t in computing_time_jenga]),
    pd.DataFrame([ { "type": "DuckDB", "timespan": t} for t in duck_computing_time]),
])
df_results

In [ ]:
fig = sns.catplot(data=df_results, x="type", y="timespan", kind="box")
fig.set_axis_labels(
    "Method of generation",
    "Duration (s)"
)
fig.set(title="Performance inserting broken characters")
fig.set(yticks=list(range(0, 121, 10))) # <--- set the ticks first
plt.ylim(0, 120)
plt.grid(axis="y")
plt.tight_layout()
plt.savefig("performance_plt.png", dpi=300)

In [14]:
r = df_results.groupby(["type"])
jenga_group = r.get_group("Jenga")
duckdb_group = r.get_group("DuckDB")
median_jenga = jenga_group["timespan"].median()
median_duckdb =  duckdb_group["timespan"].median()
std_jenga = jenga_group["timespan"].std()
std_duckdb = duckdb_group["timespan"].std()
print(f"SpeedUp: {median_jenga/median_duckdb}")
print(f"Std Jenga: {}")
print(f"Std DuckDB: {duckdb_group["timespan"].std()}")

SyntaxError: f-string: unmatched '[' (387618429.py, line 7)